#Enlace a GitHub

https://github.com/Sebastian-Cepeda-Ch/SIS421-2025-ASCC

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Instalar las bibliotecas necesarias
!pip install transformers
!pip install torch

In [ ]:
# Importar bibliotecas
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

In [ ]:
# Función para leer párrafos de los archivos
def leer_parrafos(archivo):
    with open(archivo, 'r', encoding='utf-8') as f:
        contenido = f.read()
    parrafos = [p.strip() for p in contenido.split('*') if p.strip()]
    return parrafos

In [ ]:
# Ruta al dataset
ruta = '/content/drive/MyDrive/SIS421/Laboratorios/Datasets/NLP/dataset_documentos'

In [ ]:
# Leer los archivos y asignar etiquetas
parrafos_estudiantes = leer_parrafos(ruta + '/estudiantes.txt')
etiquetas_estudiantes = [0] * len(parrafos_estudiantes)

parrafos_docentes = leer_parrafos(ruta + '/docentes.txt')
etiquetas_docentes = [1] * len(parrafos_docentes)

parrafos_administrativos = leer_parrafos(ruta + '/administrativos.txt')
etiquetas_administrativos = [2] * len(parrafos_administrativos)

parrafos_general = leer_parrafos(ruta + '/general.txt')
etiquetas_general = [3] * len(parrafos_general)

In [ ]:
# Combinar todos los párrafos y etiquetas
todos_parrafos = parrafos_estudiantes + parrafos_docentes + parrafos_administrativos + parrafos_general
todas_etiquetas = etiquetas_estudiantes + etiquetas_docentes + etiquetas_administrativos + etiquetas_general

# Dividir en conjuntos de entrenamiento y prueba
train_texts, test_texts, train_labels, test_labels = train_test_split(
    todos_parrafos, todas_etiquetas, test_size=0.2, random_state=42
)

In [ ]:
# Definir la clase personalizada para el dataset
class TextoDataset(Dataset):
    def __init__(self, textos, etiquetas, tokenizer, max_length=512):
        self.textos = textos
        self.etiquetas = etiquetas
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.textos)

    def __getitem__(self, idx):
        texto = self.textos[idx]
        etiqueta = self.etiquetas[idx]
        encoding = self.tokenizer(
            texto, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(etiqueta, dtype=torch.long)
        }

In [ ]:
# Cargar el tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Crear los datasets de entrenamiento y prueba
train_dataset = TextoDataset(train_texts, train_labels, tokenizer)
test_dataset = TextoDataset(test_texts, test_labels, tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Crear DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Cargar el modelo BERT para clasificación
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Mover el modelo a la GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Definir el optimizador y la función de pérdida
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = CrossEntropyLoss()

In [ ]:
# Función de entrenamiento manual
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc="Entrenando"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

In [ ]:
# Función de evaluación manual
def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluando"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    return total_loss / len(test_loader), accuracy

In [ ]:
# Entrenamiento y evaluación manual
from pathlib import Path
import torch

# Directorio para guardar checkpoints (añade esto al inicio del código)
CHECKPOINT_DIR = "/content/drive/MyDrive/SIS421/Laboratorios/Datasets/NLP"
Path(CHECKPOINT_DIR).mkdir(parents=True, exist_ok=True)  # Crea la carpeta si no existe

num_epochs = 3
best_test_loss = float('inf') # Para guardar el mejor modelo

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    # Entrenamiento y evaluación

    train_loss = train(model, train_loader, optimizer, criterion, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)

    # --- CHECKPOINT: Guardar modelo cada época o cuando mejore la pérdida ---
    checkpoint_path = f"{CHECKPOINT_DIR}/checkpoint_epoch_{epoch+1}.pt"

    #Guardar solo si el modelo mejora (recomendado)
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'test_loss': test_loss,
            'test_accuracy': test_accuracy,
        }, f"{CHECKPOINT_DIR}/best_model.pt")  # Guarda solo el mejor

    print(f"Checkpoint guardado en {checkpoint_path}")
    print(f"Pérdida de entrenamiento: {train_loss:.4f}")
    print(f"Pérdida de prueba: {test_loss:.4f}, Precisión en prueba: {test_accuracy:.4f}")


Epoch 1/3


Entrenando:   8%|▊         | 5/63 [04:55<54:01, 55.89s/it]

In [ ]:
# Definir las clases
CLASES = {
    0: "Estudiantes",
    1: "Docentes",
    2: "Administrativos",
    3: "General"
}
def inferir_clase(texto, model_path, tokenizer_path, max_length=512):
    """
    Función para inferir la clase de un texto dado usando un modelo BERT entrenado.

    Args:
        texto (str): El texto a clasificar.
        model_path (str): Ruta al directorio donde se guardó el modelo entrenado.
        tokenizer_path (str): Ruta al directorio donde se guardó el tokenizador.
        max_length (int): Longitud máxima de la secuencia para el tokenizador.

    Returns:
        str: La clase predicha del texto.
    """
    # Cargar el tokenizador y el modelo
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    model = BertForSequenceClassification.from_pretrained(model_path)

    # Mover el modelo a la GPU si está disponible
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Tokenizar el texto
    encoding = tokenizer(
        texto,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

    # Mover los tensores al dispositivo adecuado
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Realizar la inferencia
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        prediccion = torch.argmax(logits, dim=1).item()

    # Devolver la clase predicha
    return CLASES[prediccion]

In [ ]:
#Guardar el modelo entrenado (opcional)
model.save_pretrained('/content/drive/MyDrive/SIS421/Laboratorios/Datasets/NLP/dataset_documentos/bert_model_manual')
tokenizer.save_pretrained('/content/drive/MyDrive/SIS421/Laboratorios/Datasets/NLP/dataset_documentos/bert_tokenizer_manual')

('/content/drive/MyDrive/SIS421/Laboratorios/Datasets/NLP/dataset_documentos/bert_tokenizer_manual/tokenizer_config.json',
 '/content/drive/MyDrive/SIS421/Laboratorios/Datasets/NLP/dataset_documentos/bert_tokenizer_manual/special_tokens_map.json',
 '/content/drive/MyDrive/SIS421/Laboratorios/Datasets/NLP/dataset_documentos/bert_tokenizer_manual/vocab.txt',
 '/content/drive/MyDrive/SIS421/Laboratorios/Datasets/NLP/dataset_documentos/bert_tokenizer_manual/added_tokens.json')

In [ ]:
model_path = '/content/drive/MyDrive/SIS421/Laboratorios/Datasets/NLP/dataset_documentos/bert_model_manual'
tokenizer_path = '/content/drive/MyDrive/SIS421/Laboratorios/Datasets/NLP/dataset_documentos/bert_tokenizer_manual'
texto_ejemplo = "Profundizar conocimiento de actualidad socialmente útiles de su profesión para la solución de problemas técnicos consustanciados a la investigación científica básica y aplicada."

clase_predicha = inferir_clase(texto_ejemplo, model_path, tokenizer_path)
print(f"El texto pertenece a la clase: {clase_predicha}")

El texto pertenece a la clase: Administrativos


In [ ]:
import matplotlib.pyplot as plt

# Listas para almacenar métricas
train_losses = []
test_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)

    # Guardar métricas
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    # Guardar checkpoint (opcional)
    ...

# Gráfica de pérdida
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()

# Gráfica de precisión
plt.subplot(1, 2, 2)
plt.plot(test_accuracies, label='Test Accuracy', color='green')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()
plt.show()